In [ ]:
%pip install -qU pymupdf openai ics ipywidgets python-dotenv

In [ ]:
# Get OpenAI API key from .env file
from dotenv import load_dotenv
load_dotenv()
import os
openai_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Import necessary libraries
import fitz  # PyMuPDF, for PDF parsing
import openai  # For OpenAI API (if required)
from ics import Calendar, Event
from datetime import datetime, timedelta
import re
from IPython.display import display
import ipywidgets as widgets



In [ ]:
# Step 1: Prompt user to upload a PDF file with flight information
file_upload = widgets.FileUpload(accept='.pdf', multiple=False)
display(file_upload)


In [ ]:

# Function to save uploaded file
def save_uploaded_file(uploaded_file):
    file_name = list(uploaded_file.keys())[0]
    with open(file_name, 'wb') as f:
        f.write(uploaded_file[file_name]['content'])
    return file_name

# Wait for file upload
while not file_upload.value:
    pass
pdf_file_name = save_uploaded_file(file_upload.value)

# Step 2: Parse PDF for flight information
def extract_text_from_pdf(pdf_file):
    text = ""
    with fitz.open(pdf_file) as pdf:
        for page in pdf:
            text += page.get_text()
    return text

pdf_text = extract_text_from_pdf(pdf_file_name)

# Step 3: Use OpenAI model to parse text if necessary
# Define the prompt to pass to OpenAI's model
def parse_flight_info(text):
    openai.api_key = openai_key
    prompt = (
        "Extract detailed flight segment information from the following text. "
        "For each flight segment, provide the airline, flight number, departure date and time, "
        "arrival date and time, departure airport, and arrival airport.\n\n"
        f"{text}"
    )
    response = openai.Completion.create(
        engine="gpt-4o-mini",
        prompt=prompt,
        max_tokens=1500
    )
    return response.choices[0].text.strip()

parsed_info = parse_flight_info(pdf_text)
print("Parsed Flight Information:\n", parsed_info)

# Step 4: Generate .ics files for each flight segment
def generate_ics_from_parsed_info(parsed_info):
    # Split flight information into individual segments if necessary
    segments = parsed_info.split("Segment")  # assuming the model separates segments
    for i, segment in enumerate(segments):
        # Extract relevant details for each segment
        match = re.search(
            r"Airline:\s*(.*?)\nFlight Number:\s*(.*?)\nDeparture Date:\s*(.*?)\nDeparture Time:\s*(.*?)\n"
            r"Arrival Date:\s*(.*?)\nArrival Time:\s*(.*?)\nDeparture Airport:\s*(.*?)\nArrival Airport:\s*(.*?)",
            segment,
            re.DOTALL,
        )
        if not match:
            continue  # Skip if no match is found

        airline, flight_number, dep_date, dep_time, arr_date, arr_time, dep_airport, arr_airport = match.groups()

        # Parse dates and times
        departure = datetime.strptime(f"{dep_date} {dep_time}", "%Y-%m-%d %H:%M")
        arrival = datetime.strptime(f"{arr_date} {arr_time}", "%Y-%m-%d %H:%M")

        # Create calendar event for the flight segment
        cal = Calendar()
        event = Event()
        event.name = f"{airline} Flight {flight_number}"
        event.begin = departure.isoformat()
        event.end = arrival.isoformat()
        event.location = f"{dep_airport} to {arr_airport}"
        event.description = f"Flight {flight_number} with {airline} from {dep_airport} to {arr_airport}."

        # Add event to calendar and save as .ics
        cal.events.add(event)
        with open(f"flight_segment_{i+1}.ics", "w") as f:
            f.writelines(cal.serialize_iter())

        print(f".ics file created for flight segment {i+1}")

# Run the function to generate .ics files
generate_ics_from_parsed_info(parsed_info)
